In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from PIL import Image

In [2]:
train_path = 'E:/Final Year project/Classification model/DATASET/DATASET/TRAIN'
test_path = 'E:/Final Year project/Classification model/DATASET/DATASET/TEST'
IMG_BREDTH = 30
IMG_HEIGHT = 60
num_classes = 2

In [3]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['O', 'R'], 
                                                                          batch_size=100)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['O', 'R'], 
                                                      batch_size=100)

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [4]:
def cnn_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT,IMG_BREDTH,3)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
              
    model.summary()
              
    return model

def use_model(path):
    
    model = load_model('best_waste_classifier.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
    
#     code using PIL
#     model = load_model('best_waste_classifier.h5')
#     pic1 = plt.imread(path)
#     pic = Image.open(path).resize((IMG_BREDTH, IMG_HEIGHT))
#     plt.imshow(pic1)
#     if model.predict_classes(np.expand_dims(pic, axis=0)) == 0:
#         classes = 'ORGANIC'
#     elif model.predict_classes(np.expand_dims(pic, axis=0)) == 1:
#         classes = 'RECYCLABLE'
    
    return classes

In [5]:
model = cnn_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 28, 32)        9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 26, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 13, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 11, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 9, 64)         3

In [6]:
checkpoint = ModelCheckpoint('best_waste_classifier.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

In [7]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1.0e-4), metrics=['accuracy'])

In [8]:
# run code to train the neural network

model = model.fit_generator(train_batch,  
                            validation_data=test_batch,  
                            epochs=100, 
                            verbose=1, 
                            callbacks=[checkpoint])

e:\cemtrex labs\open images\data\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
226/226 [==============================] - 3440s 15s/step - loss: 2.1246 - accuracy: 0.6428 - val_loss: 0.3716 - val_accuracy: 0.8639
Epoch 2/100
226/226 [==============================] - 205s 905ms/step - loss: 0.4791 - accuracy: 0.7833 - val_loss: 0.3593 - val_accuracy: 0.8516
Epoch 3/100
226/226 [==============================] - 201s 889ms/step - loss: 0.4582 - accuracy: 0.7943 - val_loss: 0.3866 - val_accuracy: 0.8404
Epoch 4/100
226/226 [==============================] - 214s 948ms/step - loss: 0.4461 - accuracy: 0.7992 - val_loss: 0.3726 - val_accuracy: 0.8524
Epoch 5/100
226/226 [==============================] - 218s 965ms/step - loss: 0.4315 - accuracy: 0.8052 - val_loss: 0.3616 - val_accuracy: 0.8563
Epoch 6/100
226/226 [==============================] - 173s 764ms/step - loss: 0.4332 - accuracy: 0.8094 - val_loss: 0.3338 - val_accuracy: 0.8583
Epoch 7/100
226/226 [==============================] - 162s 715ms/step - loss: 0.4289 - accuracy: 0.8093 - val_loss: 0.

226/226 [==============================] - 156s 691ms/step - loss: 0.3273 - accuracy: 0.8615 - val_loss: 0.3778 - val_accuracy: 0.8480
Epoch 57/100
226/226 [==============================] - 155s 686ms/step - loss: 0.3387 - accuracy: 0.8581 - val_loss: 0.3594 - val_accuracy: 0.8492
Epoch 58/100
226/226 [==============================] - 154s 679ms/step - loss: 0.3395 - accuracy: 0.8570 - val_loss: 0.3278 - val_accuracy: 0.8615
Epoch 59/100
226/226 [==============================] - 153s 676ms/step - loss: 0.3361 - accuracy: 0.8591 - val_loss: 0.3438 - val_accuracy: 0.8556
Epoch 60/100
226/226 [==============================] - 153s 675ms/step - loss: 0.3315 - accuracy: 0.8614 - val_loss: 0.3020 - val_accuracy: 0.8774
Epoch 61/100
226/226 [==============================] - 153s 677ms/step - loss: 0.3269 - accuracy: 0.8672 - val_loss: 0.3233 - val_accuracy: 0.8619
Epoch 62/100
226/226 [==============================] - 153s 676ms/step - loss: 0.3322 - accuracy: 0.8620 - val_loss: 0.3245 

In [30]:
answer = use_model('E:/Final Year project/Images/R_22.jpg')
print(answer)

[1]


In [31]:
if answer == [0]:
    print("organic")
else:
    print("recyclable")

recyclable
